In [1]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os

load_dotenv()
client = OpenAI()

def get_australian_barcode_via_api(product_name, api_client):
    if not isinstance(product_name, str) or not product_name.strip():
        return "无效的产品名称"

    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that provides Australian product barcodes. When asked for a barcode, return only the numerical barcode number or 'Not Found'. If you cannot find it, return 'Not Found'.",
        },
        {
            "role": "user",
            "content": f"What is the Australian barcode for the product '{product_name}'?",
        }
    ]
    try:
        completion = api_client.chat.completions.create(
            model="gpt-4o-mini-search-preview",  # 您可以根据需要更换模型
            messages=messages,
        )
        barcode = completion.choices[0].message.content.strip()
        return barcode
    except Exception as e:
        print(f"调用API获取 '{product_name}' 的条形码时出错: {e}")
        return "API调用失败"

def main():
    if not client or not client.api_key or client.api_key.startswith("sk-xxx"):
        print("API客户端未正确配置，程序即将退出。")
        return

    excel_file = 'coles_data.xlsx'
    if not os.path.exists(excel_file):
        print(f"错误: 文件 '{excel_file}' 不存在。请确保文件与脚本在同一目录下。")
        return

    try:
        df = pd.read_excel(excel_file)
        if '产品名称' not in df.columns:
            print(f"错误: Excel文件 '{excel_file}' 中缺少 '产品名称' 列。")
            return
    except Exception as e:
        print(f"读取Excel文件 '{excel_file}' 时出错: {e}")
        return
    product_names = df['产品名称'].tolist()
    barcodes = [None] * len(product_names)
    MAX_WORKERS = 20

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_index = {
            executor.submit(get_australian_barcode_via_api, name, client): i 
            for i, name in enumerate(product_names)
        }
        for future in tqdm(future_to_index.keys(), total=len(product_names), desc="正在获取条形码"):
            index = future_to_index[future]
            try:
                barcodes[index] = future.result()
            except Exception as e:
                product_name = product_names[index]
                print(f"处理 '{product_name}' 时发生意外错误: {e}")
                barcodes[index] = "处理异常"

    df['条形码'] = barcodes
    output_file_path = excel_file.replace('.xlsx', '_with_barcodes.xlsx')
    try:
        df.to_excel(output_file_path, index=False, engine='openpyxl')
        print("\n" + "="*50)
        print(f"🎉 文件处理完成！")
        print(f"已将条形码成功写入到 '{output_file_path}'。")
        print("="*50)
    except Exception as e:
        print(f"保存到Excel文件 '{output_file_path}' 时出错: {e}")

if __name__ == "__main__":
    main()

正在获取条形码: 100%|██████████| 1312/1312 [03:51<00:00,  5.66it/s]


🎉 文件处理完成！
已将条形码成功写入到 'coles_data_with_barcodes.xlsx'。


In [2]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os

load_dotenv()
client = OpenAI()

def get_australian_barcode_via_api(product_name, api_client):
    if not isinstance(product_name, str) or not product_name.strip():
        return "无效的产品名称"

    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that provides Australian product barcodes. When asked for a barcode, return only the numerical barcode number or 'Not Found'. If you cannot find it, return 'Not Found'.",
        },
        {
            "role": "user",
            "content": f"What is the Australian barcode for the product '{product_name}'?",
        }
    ]
    try:
        completion = api_client.chat.completions.create(
            model="gpt-4o-mini-search-preview",
            messages=messages,
        )
        barcode = completion.choices[0].message.content.strip()
        return barcode
    except Exception as e:
        print(f"调用API获取 '{product_name}' 的条形码时出错: {e}")
        return "API调用失败"

def main():
    if not client or not client.api_key or client.api_key.startswith("sk-xxx"):
        print("API客户端未正确配置，程序即将退出。")
        return

    excel_file = 'woolworths_data.xlsx'
    if not os.path.exists(excel_file):
        print(f"错误: 文件 '{excel_file}' 不存在。请确保文件与脚本在同一目录下。")
        return

    try:
        df = pd.read_excel(excel_file)
        if '产品名称' not in df.columns:
            print(f"错误: Excel文件 '{excel_file}' 中缺少 '产品名称' 列。")
            return
    except Exception as e:
        print(f"读取Excel文件 '{excel_file}' 时出错: {e}")
        return
    product_names = df['产品名称'].tolist()
    barcodes = [None] * len(product_names)
    MAX_WORKERS = 20 

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_index = {
            executor.submit(get_australian_barcode_via_api, name, client): i 
            for i, name in enumerate(product_names)
        }
        for future in tqdm(future_to_index.keys(), total=len(product_names), desc="正在获取条形码"):
            index = future_to_index[future]
            try:
                barcodes[index] = future.result()
            except Exception as e:
                product_name = product_names[index]
                print(f"处理 '{product_name}' 时发生意外错误: {e}")
                barcodes[index] = "处理异常"

    df['条形码'] = barcodes
    output_file_path = excel_file.replace('.xlsx', '_with_barcodes.xlsx')
    try:
        df.to_excel(output_file_path, index=False, engine='openpyxl')
        print("\n" + "="*50)
        print(f"🎉 文件处理完成！")
        print(f"已将条形码成功写入到 '{output_file_path}'。")
        print("="*50)
    except Exception as e:
        print(f"保存到Excel文件 '{output_file_path}' 时出错: {e}")

if __name__ == "__main__":
    main()

正在获取条形码: 100%|██████████| 1279/1279 [03:51<00:00,  5.54it/s]


🎉 文件处理完成！
已将条形码成功写入到 'woolworths_data_with_barcodes.xlsx'。


In [3]:
import pandas as pd
import re
import os

def clean_barcode(barcode_data):
    """
    清理条形码数据。
    如果数据是字符串，则尝试用正则表达式提取12或13位的数字条形码。
    如果找不到匹配项，或者数据不是字符串（例如，已经是数字或空值），则返回原数据。
    """
    # 检查输入是否为字符串，如果不是则直接返回
    if not isinstance(barcode_data, str):
        return barcode_data

    # 正则表达式模式：匹配一个由12或13位数字组成的完整单词
    # \b 是单词边界，确保我们不会匹配一个更长数字的一部分
    pattern = re.compile(r'\b(\d{12,13})\b')
    
    # 在字符串中搜索该模式
    match = pattern.search(barcode_data)
    
    # 如果找到了匹配项，返回第一个匹配到的分组（即纯数字条形码）
    if match:
        return match.group(1)
    else:
        return "Not Found"

def main():
    """
    主函数，用于读取Excel，清理条形码列，并保存到新文件。
    """
    # 定义输入和输出文件名
    input_file = 'coles_data_with_barcodes.xlsx'
    output_file = 'coles_data_with_barcodes_cleaned.xlsx'

    # 检查输入文件是否存在
    if not os.path.exists(input_file):
        print(f"❌ 错误: 输入文件 '{input_file}' 不存在。")
        print("请确保您已经运行了第一个脚本，并生成了此文件。")
        return

    print(f"🚀 开始处理文件: '{input_file}'...")

    try:
        # 读取Excel文件
        df = pd.read_excel(input_file)
        
        # 检查“条形码”列是否存在
        if '条形码' not in df.columns:
            print(f"❌ 错误: Excel文件 '{input_file}' 中缺少 '条形码' 列。")
            return
            
        print("🔍 正在应用新的清理规则清理“条形码”列...")
        
        # 使用.apply()方法将更新后的清理函数应用到“条形码”列的每一个单元格
        df['条形码'] = df['条形码'].apply(clean_barcode)
        
        # 将清理后的DataFrame保存到新的Excel文件
        df.to_excel(output_file, index=False, engine='openpyxl')
        
        print("\n" + "="*50)
        print(f"🎉 清理完成！")
        print(f"已将处理后的数据保存到新文件: '{output_file}'")
        print("="*50)

    except Exception as e:
        print(f"处理过程中发生错误: {e}")

if __name__ == "__main__":
    main()

🚀 开始处理文件: 'coles_data_with_barcodes.xlsx'...
🔍 正在应用新的清理规则清理“条形码”列...

🎉 清理完成！
已将处理后的数据保存到新文件: 'coles_data_with_barcodes_cleaned.xlsx'


In [4]:
import pandas as pd
import re
import os

def clean_barcode(barcode_data):
    """
    清理条形码数据。
    如果数据是字符串，则尝试用正则表达式提取12或13位的数字条形码。
    如果找不到匹配项，或者数据不是字符串（例如，已经是数字或空值），则返回原数据。
    """
    # 检查输入是否为字符串，如果不是则直接返回
    if not isinstance(barcode_data, str):
        return barcode_data

    # 正则表达式模式：匹配一个由12或13位数字组成的完整单词
    # \b 是单词边界，确保我们不会匹配一个更长数字的一部分
    pattern = re.compile(r'\b(\d{12,13})\b')
    
    # 在字符串中搜索该模式
    match = pattern.search(barcode_data)
    
    # 如果找到了匹配项，返回第一个匹配到的分组（即纯数字条形码）
    if match:
        return match.group(1)
    else:
        return "Not Found"

def main():
    """
    主函数，用于读取Excel，清理条形码列，并保存到新文件。
    """
    # 定义输入和输出文件名
    input_file = 'woolworths_data_with_barcodes.xlsx'
    output_file = 'woolworths_data_with_barcodes_cleaned.xlsx'

    # 检查输入文件是否存在
    if not os.path.exists(input_file):
        print(f"❌ 错误: 输入文件 '{input_file}' 不存在。")
        print("请确保您已经运行了第一个脚本，并生成了此文件。")
        return

    print(f"🚀 开始处理文件: '{input_file}'...")

    try:
        # 读取Excel文件
        df = pd.read_excel(input_file)
        
        # 检查“条形码”列是否存在
        if '条形码' not in df.columns:
            print(f"❌ 错误: Excel文件 '{input_file}' 中缺少 '条形码' 列。")
            return
            
        print("🔍 正在应用新的清理规则清理“条形码”列...")
        
        # 使用.apply()方法将更新后的清理函数应用到“条形码”列的每一个单元格
        df['条形码'] = df['条形码'].apply(clean_barcode)
        
        # 将清理后的DataFrame保存到新的Excel文件
        df.to_excel(output_file, index=False, engine='openpyxl')
        
        print("\n" + "="*50)
        print(f"🎉 清理完成！")
        print(f"已将处理后的数据保存到新文件: '{output_file}'")
        print("="*50)

    except Exception as e:
        print(f"处理过程中发生错误: {e}")

if __name__ == "__main__":
    main()

🚀 开始处理文件: 'woolworths_data_with_barcodes.xlsx'...
🔍 正在应用新的清理规则清理“条形码”列...

🎉 清理完成！
已将处理后的数据保存到新文件: 'woolworths_data_with_barcodes_cleaned.xlsx'
